# OECM Benchmark Data Pipeline

The Benchmark data pipelines organize and assemble benchmark data needed for the ITR tool.  This pipeline supports the OECM Benchmark version 2 (published 4 May 2022).


### Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [1]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import numpy as np
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc
# import python_pachyderm

Define Environment and Execution Variables

In [2]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

In [3]:
import io
import json
import itertools

In [4]:
# See data-platform-demo/pint-demo.ipynb for quantify/dequantify functions

import warnings  # needed until quantile behaves better with Pint quantities in arrays
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
from openscm_units import unit_registry

# openscm_units doesn't make it easy to set preprocessors.  This is one way to do it.
unit_registry.preprocessors=[
     # lambda s1: s1.replace('passenger km', 'passenger_km'),
     lambda s2: s2.replace('BoE', 'boe'),
]

PintType.ureg = unit_registry
ureg = unit_registry
set_application_registry(ureg)
Q_ = ureg.Quantity
PA_ = PintArray

ureg.define("CO2e = CO2 = CO2eq = CO2_eq")
ureg.define("Fe = [iron] = Steel")
ureg.define("iron = Fe")
ureg.define("Al = [aluminum] = Aluminum")
ureg.define("aluminum = Al")
ureg.define("Cement = [cement]")
ureg.define("cement = Cement")

# For reports that use 10,000 t instead of 1e3 or 1e6                                                                                                                                                                                                        
ureg.define('myria- = 10000')

# These are for later
ureg.define('fraction = [] = frac')
ureg.define('percent = 1e-2 frac = pct = percentage')
ureg.define('ppm = 1e-6 fraction')

ureg.define("USD = [currency]")
ureg.define("EUR = nan USD")
ureg.define("JPY = nan USD")

ureg.define("btu = Btu")
ureg.define("mmbtu = 1e6 btu")
# ureg.define("boe = 5.712 GJ")                                                                                                                                                                                                                              
ureg.define("boe = 6.1178632 GJ")
ureg.define("mboe = 1e3 boe")
ureg.define("mmboe = 1e6 boe")

# Transportation activity

ureg.define("vehicle = [vehicle] = v")
ureg.define("passenger = [passenger] = p = pass")
ureg.define("vkm = vehicle * kilometer")
ureg.define("pkm = passenger * kilometer")
ureg.define("tkm = tonne * kilometer")

ureg.define('hundred = 1e2')
ureg.define('thousand = 1e3')
ureg.define('million = 1e6')
ureg.define('billion = 1e9')
ureg.define('trillion = 1e12')
ureg.define('quadrillion = 1e15')

### S3 and boto3

In [5]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

### Connecting to Trino with sqlalchemy

In the context of the Data Vault, this pipeline operates with full visibiilty into all the data it prepares for the ITR tool.  When the data is output, it is labeled so that the Data Vault can enforce its data management access rules.

In [6]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
dera_schema = 'sandbox'
dera_prefix = 'dera_'
gleif_schema = 'sandbox'
rmi_schema = 'sandbox'
iso3166_schema = 'sandbox'
essd_schema = 'sandbox'
essd_prefix = 'essd_'
demo_schema = 'demo_dv'

engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev


### Definitions and dictionaries for reading from / writing to the outside world

In [7]:
transport_elements = ['Subsector', 'Total CO2 Emissions', 'Emission Intensity', 'Energy Intensity']
bldg_elements = ['Parameter', 'Residential Buildings', 'Commercial Buildings']

benchmark_years = pd.Series(name='Production', index=pd.Index(list(range(2019,2051))), dtype='float64')
benchmark_years.index.name = 'Year'

oecm_dict = {
    'Utilities': ['Subsector', 'Utilities', 'Total Energy Production (power + gas/fuels)', 'Total CO2 equivalent', 'PJ', 't CO2e/GJ'],
    'Steel': ['Parameter', 'Materials / Steel', 'Annual production volume- Iron & Steel Industry', 'Total CO2 equivalent', 'Mt Steel', 't CO2e/(t Steel)'],
    'Energy': ['Subsector', 'Energy Industry', 'Total FINAL ENERGY RE-FT production', 'Total CO2 equivalent', 'PJ', 't CO2e/GJ'],
    'Transport_UNPRI': ['Subsector','Road Transport', 'Road Transport (excluding vehicle manufacturing)', '', 'km', 'g CO2e/pkm'],
    'Alu': ['Parameter', 'Aluminium Industry', 'Annual production volume- aluminium Industry', 'Total CO2 equivalent', 'Mt Aluminum', 't CO2e/(t Aluminum)'],
    'Cement': ['Parameter', 'Materials / Cement', 'Cement - production volume in mega tonnes per year', 'Total CO2 equivalent', 'Mt Cement', 't CO2e/(t Cement)'],
    'Buildings': ['Parameter', 'Buildings', 'Construction: Residential and Commercial Building - Economic value', 'Total CO2 equivalent', 'billion m**2', 't CO2e/(thousand m**2)'],
    'Chemical Industry': ['Parameter', 'Chemical Industry', 'Total Chemical Industry', 'Total CO2 equivalent', 'billion USD', 'kg CO2e/USD'],
    'Tex & Lea': ['Parameter', 'Textile & Leather', 'Total Textile & Leather', 'Total CO2 equivalent', 'billion USD', 'kg CO2e/USD'],
}

itr_dict = {
    'Utilities': 'Electricity Utilities',
    'Steel': 'Steel',
    'Energy': 'Oil & Gas',
    'Transport_UNPRI': 'Autos',
    'Alu': 'Aluminum',
    'Cement': 'Cement',
    'Buildings': 'Buildings',
    'Chemical Industry': 'Chemicals',
    'Tex & Lea': 'Textiles'
}

### Interpolation Function

Production is CAGR-based; Emissions are CAGR-based if the ratio fo start/finish <= 2.

When start/finish gets too high, the curve gets a pronounced drop in the first year

When finish is zero, the curve can only approach is asymptotically, which is also problematic.
Instead, use linear interpolation when it's time to drive the curve down to zero

In [8]:
# Interpolate missing benchmark values for Production and Emissions, then compute Emissions Intensities (EI)

def interpolate_benchmark(df, first_year=2019, last_year=2050):
    # Interpolate all missing Production and Scope emissions, except Scope 3 remains zero until we change benchmarks

    i = first_year
    while i < last_year:
        idx1 = i # .Production.first_valid_index()
        idx2 = df[df.index>i].Production.first_valid_index()
        if idx2 is None:
            break

        nth_root = 1/(idx2-idx1)
        for col in ['Production', 'S1', 'S2', 'S1S2', 'S3', 'S1S2S3']:
            if df.loc[idx2, col] == 0 or (df.loc[idx1, col]/df.loc[idx2, col]).m > 2:
                # print(f"Linear: {df.loc[idx1, col].m}/{df.loc[idx2, col].m}")
                # Linear interpolation
                delta = (df.loc[idx2, col] - df.loc[idx1, col]) / (idx2-idx1)
                for j in range(idx1, idx2):
                    df.loc[j+1, col] = df.loc[j, col] + delta
            else:
                # print(f"CAGR: {df.loc[idx1, col].m}/{df.loc[idx2, col].m}")
                # CAGR interpolation
                multiplier = ((df.loc[idx2, col] / df.loc[idx1, col])**nth_root).m
                for j in range(idx1, idx2):
                    df.loc[j+1, col] = df.loc[j, col] * multiplier
        i = idx2
    df['EI_S1S2'] = (df.S1S2 / df.Production).astype(f"pint[{sector_elements[5]}]")
    df['EI_S3'] = (df.S3 / df.Production).astype(f"pint[{sector_elements[5]}]")
    df['EI_S1S2S3'] = (df.S1S2S3 / df.Production).astype(f"pint[{sector_elements[5]}]")

    # By convention, the d_ column is zero at the start of the series.
    # Subsequent values multiply the previous quantity by the present d_ number to get the present quanity
    df["d_Production"] = [0] + [m.m-1 for m in (df.Production.values[1:]/df.Production.values[:-1]).tolist()]

    return df

### Principle processing function

Start with dataframe containing "messy" data from Spreadsheet, then clean it up to a standard format

In [30]:
def process_sector_benchmark (sector_benchmark, region, sector, sector_elements):
    s = sector_benchmark.iloc[:, 1]
    # Transport_UNPRI doesn't have 'Total CO2 equivalent' in its scope strings...
    df_elements = [sector_elements[0],sector_elements[2],
                   ' '.join([f"{sector_elements[1]} - Scope 1:", sector_elements[3]]).rstrip(),
                   ' '.join([f"{sector_elements[1]} - Scope 2:", sector_elements[3]]).rstrip(),
                   ' '.join([f"{sector_elements[1]} - Scope 3:", sector_elements[3]]).rstrip()]
    
    # Hand-adjust the rows and columns we'll be processing.  A few sectors are unique in their shape/data.
    # Some sheets have extra years of data, which pushes 2050 to the right.  We allocate a generous number
    # of columns so that we capture 2050, and then we drop the columns we don't need, either from middle or the right
    if sector == 'Chemical Industry':
        df = sector_benchmark.iloc[s.loc[s.isin(df_elements).fillna(False)].index, 1:14][[True]*2 + [False]*6 + [True]*3]
    elif sector == 'Tex & Lea':
        df = sector_benchmark.iloc[s.loc[s.isin(df_elements).fillna(False)].index, 1:14][[True]*2 + [False]*4 + [True]*3]
        # There's an extra column announcing 'ok' in the Europe data we must drop
        if region == 'Europe':
            df.drop(columns=df.columns[2], inplace=True)
    else:
        df = sector_benchmark.iloc[s.loc[s.isin(df_elements).fillna(False)].index, 1:14][[True]*2 + [False]*3 + [True]*3]
        if sector == 'Transport_UNPRI':
            # There's an extra column announcing 'equals [Mt CO2]' we must drop
            df.drop(columns=df.columns[2], inplace=True)
    while df.iloc[0, -1] != '2050':
        df.drop(columns=df.columns[-1], inplace=True)
    
    # Drop empty columns and transpose so that years are in rows
    df = df.dropna(how='all', axis=1).T

    # Now ready to build the DataFrame...
    df.columns = ['Year', 'Production', 'S1', 'S2', 'S3']
    df.S3.fillna(0, inplace=True)
    units = df.iloc[1, 1:].map(lambda x: x[1:-1].split('/')[0].replace('Mt CO2 equiv.', 'Mt CO2e'), na_action='ignore')
    units.replace("bn $ GDP", "billion USD")
    units.Production = sector_elements[4]
    df = df.iloc[2:].astype({'Year':'int', 'Production':'float', 'S1':'float', 'S2':'float', 'S3':'float'}).set_index('Year')

    # Fix Transport by proportionalizing results to Autos only.  When and how to handle Freight?
    if sector == 'Transport_UNPRI':
        # Need to proportionalize total sector emissions vs. passenger-only and then feed back into total
        s = pd.concat([sector_benchmark.iloc[:8, 1],sector_benchmark.iloc[87:, 1]])
        passenger = sector_benchmark.iloc[s.loc[s.isin(transport_elements).fillna(False)].index, 1:14]
        while passenger.iloc[0, -1] != '2050':
            passenger.drop(columns=passenger.columns[-1], inplace=True)
        passenger = passenger.dropna(how='all', axis=1)[1:4].T
        passenger.columns = passenger.iloc[0]
        passenger_units = passenger.iloc[1].map(lambda x: x[1:-1].split('/')[0].replace('Mt CO2 equiv.', 'Mt CO2e'), na_action='ignore')
        for unit in passenger_units.index:
            if 'Intensity' in unit:
                passenger_units[unit] = f"{passenger_units[unit]} / pkm"
        units.Production = (ureg(passenger_units['Total CO2 Emissions']) / ureg(passenger_units['Emission Intensity'])).to('gigapkm').u
        passenger = passenger.iloc[2:].astype('float64')
        passenger.index = df.index
        # Slice out old data columns so that everything starts at 2019
        df.drop([2017,2018], inplace=True, errors='ignore')
        passenger.drop([2017,2018], inplace=True, errors='ignore')
        df = pd.concat([df, passenger], axis=1)
        df.Production = df.apply(lambda x: Q_(x['Total CO2 Emissions'], passenger_units['Total CO2 Emissions'])
                                 / Q_(x['Emission Intensity'], passenger_units['Emission Intensity']) if x['Emission Intensity'] else np.nan,
                                 axis=1).fillna(method='ffill')
        scopes = ['S1', 'S2', 'S3']
        total_co2 = df[scopes].sum(axis=1)
        for scope in scopes:
            df[scope] = (df[scope] * df['Total CO2 Emissions'] / total_co2).replace(np.nan, 0)
        df.drop(columns=transport_elements[1:], inplace=True)
        # freight = oecm_bm[sector].iloc[s.loc[s.isin(transport_elements).fillna(False)].index, 1:14].dropna(how='all', axis=1)[3:6].T
        # display(freight)
    elif sector == 'Buildings':
        bldgs = sector_benchmark.iloc[s.loc[s.isin(bldg_elements).fillna(False)].index, 1:14]
        while bldgs.iloc[0, -1] != '2050':
            bldgs.drop(columns=bldgs.columns[-1], inplace=True)
        bldgs = bldgs.dropna(how='all', axis=1).T
        bldgs.columns = bldgs.iloc[0]
        bldgs_units = bldgs.iloc[1].map(lambda x: x[1:-1])
        units.Production = ureg('billions m**2').u
        bldgs = bldgs.iloc[2:].astype('float64')
        bldgs.index = df.index
        df.Production = bldgs.sum(axis=1)

    # Now insert all the missing years we need to interpolate
    df = pd.DataFrame(benchmark_years).combine_first(df)
    # Change type at the end, as the addition of np.nan values can mess with the dtype (making it dtype 'object')
    for col in df.columns:
        df[col] = df[col].astype(f"pint[{units[col]}]")
    df.insert(0, "Sector", sector_elements[1])
    df.insert(0, "Region", region)
    df['S1S2'] = df.S1 + df.S2
    df['S1S2S3'] = df.S1 + df.S2 + df.S3
    return interpolate_benchmark(df)

### Construct JSON benchmark structures

1.  Load Regional Workbook
2.  Process each Sector in the Workbook
3.  Convert resulting dataframe to dictionary structure
4.  Merge each Region/Sector dictionary into main benchmark dictionary

Note that we use linear interpolation when the overall interpolation is more than a 2:1 ratio start to finish
CAGR gets wonky both as the endpoint approaches zero (ratio becomes infinite); but it's also funky when slope is steep (though not infinitely steep)

In [31]:
production_bm = {
    "benchmark_temperature": "1.5 delta_degC",
    "benchmark_global_budget": "396 Gt CO2",
    "is_AFOLU_included": 'false',
}

ei_bm = {
    "benchmark_temperature": "1.5 delta_degC",
    "benchmark_global_budget": "396 Gt CO2",
    "is_AFOLU_included": 'false',
}

region_dict = {'Global':'OECM_Global_2022_04_22_Results',
               'Europe':'OECM_OECD_Europe_2022_04_22_results',
               'North America':'OECM_OECD_North_America_2022_04_22_results_0'}

def merge_bm_dicts(main, new):
    for scope in new.keys():
        if not main.get(scope):
            main[scope] = { 'benchmarks': [] }
        main[scope]['benchmarks'].append(new[scope]['benchmarks'][0])

for region, filename in region_dict.items():
    wb = pd.read_excel(os.environ.get('PWD')+f"/itr-data-pipeline/data/external/OECM 20220504/{filename}.xlsx", sheet_name=None, dtype=str)
    for sheet, df in wb.items():
        wb[sheet] = df.applymap(lambda x: x.rstrip(), na_action='ignore')
    for sector, sector_elements in oecm_dict.items():
        print(f"Region {region} Sector {sector}")
        df = process_sector_benchmark (wb[sector], region, sector, sector_elements)
        # It's tempting to concatenate these DataFrames, but doing so wrecks the nice PintArrays created for Production and EI
        # So instead, build up the respective dictionaries with each dataframe we process
        
        # Production is not conditioned on scope--we shouldn't even need it!
        new_prod_bm = {
            scope: {
                "benchmarks": [
                    {
                        "sector": itr_dict[sector],
                        "region": region,
                        "benchmark_metric": { "units": "dimensionless" },
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "unit": "dimensionless",
                        "projections": [
                            {
                                "year": year,
                                "value": value
                            }
                            for year, value in zip(df.index, df.d_Production)
                        ]
                    }
                ]
            }
            for scope in ['S1S2']
        }
        merge_bm_dicts(production_bm, new_prod_bm)

        bm_ei_scopes = {
            scope: {
                "benchmarks": [
                    {
                        "sector": itr_dict[sector],
                        "region": region,
                        "benchmark_metric": { "units": sector_elements[5] },
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "unit": sector_elements[5],
                        "projections": [
                            {
                                "year": year,
                                "value": value.m
                            }
                            for year, value in zip(df.index, df[f"EI_{scope}"])
                        ]
                    }
                ]
            }
            for scope in ['S1S2', 'S1S2S3']
        }

        if df.S3.sum().m:
            bm_ei_scopes['S3'] = {
                "benchmarks": [
                    {
                        "sector": itr_dict[sector],
                        "region": region,
                        "benchmark_metric": { "units": sector_elements[5] },
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "unit": sector_elements[5],
                        "projections": [
                            {
                                "year": year,
                                "value": value.m
                            }
                            for year, value in zip(df.index, df.EI_S3)
                        ]
                    }
                ]
            }

        merge_bm_dicts(ei_bm, bm_ei_scopes)


Region Global Sector Utilities
Region Global Sector Steel
Region Global Sector Energy
Region Global Sector Transport_UNPRI
Region Global Sector Alu
Region Global Sector Cement
Region Global Sector Buildings


/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


Region Global Sector Chemical Industry
Region Global Sector Tex & Lea
Region Europe Sector Utilities
Region Europe Sector Steel
Region Europe Sector Energy
Region Europe Sector Transport_UNPRI


/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


Region Europe Sector Alu
Region Europe Sector Cement
Region Europe Sector Buildings
Region Europe Sector Chemical Industry
Region Europe Sector Tex & Lea
Region North America Sector Utilities
Region North America Sector Steel
Region North America Sector Energy
Region North America Sector Transport_UNPRI
Region North America Sector Alu
Region North America Sector Cement


/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


Region North America Sector Buildings
Region North America Sector Chemical Industry
Region North America Sector Tex & Lea


### Emit Sector Benchmark Data

In [32]:
with open('production_bm.json', 'w') as f:
    print(json.dumps(production_bm, sort_keys=True, indent=4), file=f)

with open('ei_bm.json', 'w') as f:
    print(json.dumps(ei_bm, sort_keys=True, indent=4), file=f)